In [5]:
!pip install -q tensorflow torch numpy matplotlib torchvision scikit-learn


In [6]:
import sys
import os
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import json
from torchvision import transforms
from torchvision.models import resnet50
from sklearn.decomposition import PCA
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.datasets import mnist

# --- CORREÇÃO DE CAMINHO ---
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# --- OTIMIZAÇÃO DE MEMÓRIA DA GPU ---
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    print(f"Otimização de memória (Memory Growth) ativada para {len(gpus)} GPU(s).")
  except RuntimeError as e:
    print(e)

# --- IMPORTS DO PROJETO ---
from config.training_config import CosFaceConfig, BaselineConfig, InferenceConfig
from src.losses.margin_losses import CosFace

print("Setup completo. Módulos e configurações carregados.")

2025-09-19 14:17:58.777937: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Setup completo. Módulos e configurações carregados.


In [7]:
class BaselineResNet50(nn.Module):
    def __init__(self, num_classes, embedding_size=512):
        super(BaselineResNet50, self).__init__()
        self.backbone = resnet50(weights=None)
        backbone_output_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        self.embedding_layer = nn.Linear(backbone_output_features, embedding_size)
        self.classifier = nn.Linear(embedding_size, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        embedding = self.embedding_layer(x)
        return embedding

print("Arquitetura do modelo Baseline definida.")

Arquitetura do modelo Baseline definida.


In [8]:
# Carrega as configurações
cosface_cfg = CosFaceConfig()
baseline_cfg = BaselineConfig()
inference_cfg = InferenceConfig()

# Carrega Dados
(_, _), (X_test, y_test) = mnist.load_data()
sample_size = inference_cfg.SAMPLE_SIZE

# --- Processa Features do CosFace ---
print("Processando modelo CosFace...")
X_test_keras = X_test[:sample_size, ..., np.newaxis].astype('float32') / 255
cosface_model = load_model(cosface_cfg.CHECKPOINT_PATH, custom_objects={'CosFace': CosFace})
feature_extractor = Model(inputs=cosface_model.input[0], outputs=cosface_model.layers[-3].output)
cosface_features = feature_extractor.predict(X_test_keras, verbose=0)
cosface_features /= np.linalg.norm(cosface_features, axis=1, keepdims=True)

# --- Processa Features do Baseline ---
print("Processando modelo Baseline...")
preprocess_pytorch = transforms.Compose([
    transforms.ToPILImage(), transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
X_test_pytorch = torch.stack([preprocess_pytorch(img) for img in X_test[:sample_size]])

baseline_model = BaselineResNet50(num_classes=baseline_cfg.NUM_CLASSES, embedding_size=baseline_cfg.EMBEDDING_SIZE)
baseline_model.load_state_dict(torch.load(baseline_cfg.CHECKPOINT_PATH, map_location=torch.device('cpu'))['model_state_dict'])
baseline_model.eval()
with torch.no_grad():
    baseline_features_512d = baseline_model(X_test_pytorch).numpy()

# --- Redução de Dimensionalidade (PCA) ---
print("Reduzindo dimensionalidade do Baseline...")
pca = PCA(n_components=3)
baseline_features_3d = pca.fit_transform(baseline_features_512d)
baseline_features_3d /= np.linalg.norm(baseline_features_3d, axis=1, keepdims=True)

# --- Salva Embeddings em JSON ---
print("Salvando amostra de embeddings em formato JSON...")
y_test_sampled = y_test[:sample_size]
embeddings_data = [{'id': i, 'label': int(y_test_sampled[i]), 'cosface_embedding': cosface_features[i].tolist(), 'baseline_embedding_3d': baseline_features_3d[i].tolist()} for i in range(sample_size)]
json_output_path = 'deliverables/embeddings.json'
os.makedirs(os.path.dirname(json_output_path), exist_ok=True)
with open(json_output_path, 'w') as f:
    json.dump(embeddings_data, f, indent=4)
print(f"Embeddings salvos em: {json_output_path}")

# --- Gera Gráfico Comparativo ---
print("Gerando visualização...")
fig = plt.figure(figsize=(20, 9))
plt.suptitle('Comparação do Espaço de Características: Baseline vs. CosFace', fontsize=16)

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
for i in range(10):
    idx = y_test_sampled == i
    ax1.scatter(baseline_features_3d[idx, 0], baseline_features_3d[idx, 1], baseline_features_3d[idx, 2], label=f'Dígito {i}', alpha=0.4)
ax1.set_title('Modelo Baseline (ResNet50 + Softmax)')
ax1.legend()

ax2 = fig.add_subplot(1, 2, 2, projection='3d')
for i in range(10):
    idx = y_test_sampled == i
    ax2.scatter(cosface_features[idx, 0], cosface_features[idx, 1], cosface_features[idx, 2], label=f'Dígito {i}', alpha=0.4)
ax2.set_title('Modelo Otimizado (VGG8 + CosFace)')
ax2.legend()

os.makedirs('reports/figures', exist_ok=True)
plt.savefig(inference_cfg.OUTPUT_FIGURE_PATH)
plt.show()

print(f"\nGráfico comparativo salvo em: {inference_cfg.OUTPUT_FIGURE_PATH}")

Processando modelo CosFace...


2025-09-19 14:18:11.826522: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Processando modelo Baseline...


/tmp/ipykernel_131599/1689704595.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  baseline_model.load_state_dict(torch.load(baseline_cfg.CHECKPOINT_PATH, map_location=to

FileNotFoundError: [Errno 2] No such file or directory: 'models/baseline_model_epoch_20.pth'

In [ ]:
# Célula 2: Carregamento dos Dados
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_test = X_test[:, :, :, np.newaxis].astype('float32') / 255


In [ ]:
# Célula 3: Extração de Features
model_path = '../experiments/checkpoints/mnist_vgg8_cosface_3d/model.keras'

print("Carregando o modelo CosFace...")
cosface_model = load_model(model_path, custom_objects={'CosFace': CosFace})

feature_extractor = Model(inputs=cosface_model.input[0], outputs=cosface_model.layers[-3].output)

print("Extraindo features...")
cosface_features = feature_extractor.predict(X_test, verbose=1)
cosface_features /= np.linalg.norm(cosface_features, axis=1, keepdims=True)


In [ ]:
# Célula 4: Geração e Salvamento do Gráfico
print("Gerando visualização 3D...")
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

for c in range(len(np.unique(y_test))):
    ax.scatter(cosface_features[y_test==c, 0], 
               cosface_features[y_test==c, 1], 
               cosface_features[y_test==c, 2], 
               label=f'Dígito {c}',
               alpha=0.2)

ax.set_title('Visualização 3D das Features - CosFace')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Feature 3')
ax.legend()

output_path = '../assets/cosface_visualization_notebook.png'
plt.savefig(output_path)
print(f"Gráfico salvo com sucesso como '{output_path}'!")
plt.show() # Exibe o gráfico no notebook também
